# Entity Framework Examples

get nuget dependencies

In [1]:
#r "nuget: Microsoft.EntityFrameworkCore"
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite"
#r "nuget: Microsoft.EntityFrameworkCore.Proxies"
#pragma warning disable CS1701 // Assuming assembly reference matches identity 
#nullable enable


Installed Packages Microsoft.EntityFrameworkCore, 9.0.1 Microsoft.EntityFrameworkCore.Proxies, 9.0.1 Microsoft.EntityFrameworkCore.Sqlite, 9.0.1

In [3]:
using Microsoft.EntityFrameworkCore;
using Microsoft.EntityFrameworkCore.Sqlite;
using Microsoft.EntityFrameworkCore.Proxies;

The Venue Entity

In [4]:
public class Event
{
    public Event() : this(0, "", "", new Venue())
    { }
    public Event(int id, string name, string description, Venue venue)
    {
        Id = id;
        Name = name;
        Description = description;
        Venue = venue;
    }
    public int Id { get; set; }
    public string Name { get; set; }
    public string Description { get; set; }
    public virtual Venue Venue { get; set; }
}

public class Venue
{
    public Venue() :this ("", "", 0)
    { }
    public Venue(string name, string location, int capacity)
    {
        Name = name;
        Location = location;
        Capacity = capacity;
    }
    public int Id { get; set; }
    public string Name { get; set; }
    public string Location { get; set; }
    public int Capacity { get; set; }
    public virtual ICollection<Event> Events { get; set; } = new List<Event>();
}



The DbContext

In [5]:
public class TixDbContext : DbContext
{
    public TixDbContext()
    { }
    public TixDbContext(DbContextOptions<TixDbContext> options)
        : base(options) 
    { }   

    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
    {
        optionsBuilder.UseLazyLoadingProxies();
        optionsBuilder.UseSqlite("Data Source=tix.db");
    }

    public DbSet<Venue> Venue { get; set; } = null!;
}


Create some records

In [6]:
public void CreateData()
{
    using TixDbContext context = new();
    context.Database.EnsureDeleted();
    context.Database.EnsureCreated();
    var v1 = new Venue ("Kyle Field", "College Station, TX", 102733);
    var v2 = new Venue ("Darrell K Royal-Texas Memorial Stadium", "Austin, TX", 100119);
    var v3 = new Venue ("Cotton Bowl", "Dallas, TX", 92700);
    var v4 = new Venue ("AT&T Stadium", "Arlington, TX", 80000);
    var v5 = new Venue ("NRG Stadium", "Houston, TX", 72000);
    context.Venue.AddRange(v1, v2, v3, v4, v5);
    
    v1.Events.Add(new Event(1, "Aggies vs. Longhorns", "The big game", v1));
    v3.Events.Add(new Event(2, "Red River Rivalry", "The big game", v3));
    

    context.SaveChanges();
}


In [7]:
CreateData()

Run to see the database

In [8]:
#r "nuget: Microsoft.DotNet.Interactive.Sqlite,*-*"

Installed Packages Microsoft.DotNet.Interactive.Sqlite, 1.0.0-beta.25070.1

Loading extension script from `C:\Users\chuck_vbrk0he\.nuget\packages\microsoft.dotnet.interactive.sqlite\1.0.0-beta.25070.1\interactive-extensions\dotnet\extension.dib`

Query SQLite databases. 
 This extension adds support for connecting to SQLite databases using the #!connect sqlite magic command. For more information, run a cell using the #!sql magic command.

In [9]:
#!connect sqlite --kernel-name tix  "Data source=tix.db"

Kernel added: #!sql-tix

In [10]:
SELECT * FROM Venue


Id,Name,Location,Capacity
1,Kyle Field,"College Station, TX",102733
2,Darrell K Royal-Texas Memorial Stadium,"Austin, TX",100119
3,Cotton Bowl,"Dallas, TX",92700
4,AT&T Stadium,"Arlington, TX",80000
5,NRG Stadium,"Houston, TX",72000


In [11]:
SELECT * FROM Event

Id,Name,Description,VenueId
1,Aggies vs. Longhorns,The big game,1
2,Red River Rivalry,The big game,3


In [12]:
SELECT * FROM sqlite_schema

type,name,tbl_name,rootpage,sql
table,Venue,Venue,2,"CREATE TABLE ""Venue"" ( ""Id"" INTEGER NOT NULL CONSTRAINT ""PK_Venue"" PRIMARY KEY AUTOINCREMENT, ""Name"" TEXT NULL, ""Location"" TEXT NULL, ""Capacity"" INTEGER NOT NULL )"
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,Event,Event,4,"CREATE TABLE ""Event"" ( ""Id"" INTEGER NOT NULL CONSTRAINT ""PK_Event"" PRIMARY KEY AUTOINCREMENT, ""Name"" TEXT NULL, ""Description"" TEXT NULL, ""VenueId"" INTEGER NULL, CONSTRAINT ""FK_Event_Venue_VenueId"" FOREIGN KEY (""VenueId"") REFERENCES ""Venue"" (""Id"") )"
index,IX_Event_VenueId,Event,5,"CREATE INDEX ""IX_Event_VenueId"" ON ""Event"" (""VenueId"")"


Get entities from the db

In [13]:
public IEnumerable<Venue> GetVenues()
{
    using TixDbContext context = new();
    return context.Venue.ToList();
}

foreach (var venue in GetVenues())
{
    Console.WriteLine($"{venue.Name} in {venue.Location} has a capacity of {venue.Capacity}");
}

Kyle Field in College Station, TX has a capacity of 102733
Darrell K Royal-Texas Memorial Stadium in Austin, TX has a capacity of 100119
Cotton Bowl in Dallas, TX has a capacity of 92700
AT&T Stadium in Arlington, TX has a capacity of 80000
NRG Stadium in Houston, TX has a capacity of 72000
